In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# Define data directories
data_dir = 'C:/Users/hafiz/AI Internship (Rex)/Task_1_06-09-23/Male and Female face dataset/Test_Validation_Train'  # Replace with the actual path to your data directory
train_dir = os.path.join(data_dir, 'Train')
test_dir = os.path.join(data_dir, 'Test')
validation_dir = os.path.join(data_dir, 'Validation')


In [2]:
# Define data split ratios
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32


In [3]:
# Function to create and compile the CNN model
def create_model(img_width, img_height, num_classes):
    model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Change to 1 output neuron
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [4]:
# Create data generators with data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators for validation and testing (no data augmentation)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)


In [5]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)


Found 3783 images belonging to 2 classes.
Found 766 images belonging to 2 classes.
Found 766 images belonging to 2 classes.


In [6]:
# Create and train the model
img_width = 150
img_height = 150
num_classes = 2  # Adjust this based on your problem
model = create_model(img_width, img_height, num_classes)

# Train the model with your data generators and other settings
epochs = 5
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping]
)


Epoch 1/5
119/119 [==============================] - 836s 7s/step - loss: 1.5469 - accuracy: 0.5778 - val_loss: 0.9288 - val_accuracy: 0.7324
Epoch 2/5
119/119 [==============================] - 847s 7s/step - loss: 0.8835 - accuracy: 0.6714 - val_loss: 0.7746 - val_accuracy: 0.6945
Epoch 3/5
119/119 [==============================] - 854s 7s/step - loss: 0.7414 - accuracy: 0.7021 - val_loss: 0.6156 - val_accuracy: 0.7715
Epoch 4/5
119/119 [==============================] - 848s 7s/step - loss: 0.6675 - accuracy: 0.7203 - val_loss: 0.5614 - val_accuracy: 0.7924
Epoch 5/5
119/119 [==============================] - 845s 7s/step - loss: 0.6392 - accuracy: 0.7256 - val_loss: 0.5216 - val_accuracy: 0.8133


In [7]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_accuracy}")


12/24 [==============>...............] - ETA: 28s - loss: 0.6256 - accuracy: 0.7760

C:\Users\hafiz\anaconda3\lib\site-packages\PIL\Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


24/24 [==============================] - 58s 2s/step - loss: 0.6082 - accuracy: 0.7807
Test Accuracy: 0.7806788682937622


In [8]:
# Import precision_score, recall_score, f1_score, and accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Make predictions on the test set
predictions = model.predict(test_generator)
predicted_classes = np.round(predictions)

# Generate a classification report
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

print(f"Test Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


24/24 [==============================] - 54s 2s/step
Test Accuracy: 0.5117493472584856
Precision: 0.5101123595505618
Recall: 0.5926892950391645
F1-Score: 0.5483091787439612
              precision    recall  f1-score   support

 Female Face       0.51      0.43      0.47       383
   Male Face       0.51      0.59      0.55       383

    accuracy                           0.51       766
   macro avg       0.51      0.51      0.51       766
weighted avg       0.51      0.51      0.51       766

